###### Imports

Packages

In [1]:
import pandas as pd
from functions import assembleDf, epochElo, epochsElo, epochG, epochsG,\
PlayerElo, winProbG, get_recent_rating_wp, get_recent_rating_rd_wp
from datetime import datetime, timedelta
import numpy as np
import sys
sys.path.append('..')
from pyglicko2.glicko2_tests import exampleCase
from pyglicko2.glicko2 import Player
import glicko2
import time

Data

In [2]:
# read in the data that will be used with the rating systems.
matches = pd.read_csv('../Data/matches_glicko2.csv',parse_dates = 
                      ['tourney_date'], infer_datetime_format = True)

In [3]:
# read in the unfiltered data
singles_matches = pd.read_csv('../Data/singles_matches_df.csv',parse_dates = ['tourney_date'], 
                      infer_datetime_format = True, low_memory = False)

###### Rating system application

Get the Elo ratings for the matches.

In [4]:
playerClasses, eloRatingsHistory = epochsElo(matches)

In [5]:
ratingsHistory_df = assembleDf(eloRatingsHistory)

In [6]:
# fill in missing values with most recent rating.  
ratingsHistory_df = ratingsHistory_df.ffill(axis=0).fillna(1500)

In [7]:
# import glicko2 ratings and rating deviations DataFrames
g2_rh = pd.read_csv('./ratings_histories_glicko2.csv', index_col = 0, 
                   parse_dates=True, dtype=np.float64)
g2_rh.columns = g2_rh.columns.astype(int)
g2_rdh = pd.read_csv('./rd_histories_glicko2.csv', index_col = 0, 
                     parse_dates=True, dtype=np.float64)
g2_rdh.columns = g2_rdh.columns.astype(int)


In [8]:
g2_players = set(g2_rdh.columns).union(g2_rh.columns)

###### Idea: filter for winner_id and loser_id in ratinghistory pre-glicko2 analysis

In [9]:
[True if tuple(p_id)[0] and tuple(p_id[1]) in g2_players else False for p_id in \
 matches[['winner_id','loser_id']]]

[False, False]

In [10]:
in_g2 = [p[0] in g2_players and p[1] in g2_players  for p in zip(matches['winner_id'],matches['loser_id'])]

In [11]:
sum(in_g2)/matches.shape[0]

0.9528939152407472

For 95% of the matches, both players are in the glicko-2 dataframes.  Later iterations on this code will determine why not 100% of the players are getting captured by epochsG.

In [12]:
def get_recent_rating_rd_wp_lambda(tourney_date, winner_id, loser_id,
                                   ratingsHistory_df, rdHistory_df):
    """Return the most recent ratings and rating deviations for the competitors as
        well as the a priori win probability for the eventual winner according to 
        the Glicko-2 rating system. **kwargs : ratingsHistory_df,rdHistory_df"""
    try:
        if not type(tourney_date) == pd._libs.tslibs.timestamps.Timestamp:
            tourney_date = pd.to_datetime(tourney_date)
        # for robustness, assuming that rd and ratings may have different timestamps
        timestamp_rh = max(ratingsHistory_df.index[ratingsHistory_df.index<=
                                                   tourney_date])
        timestamp_rd = max(rdHistory_df.index[rdHistory_df.index<=tourney_date])
        winner_rating, loser_rating = ratingsHistory_df.loc[timestamp_rh,
                                                        [winner_id,loser_id]]
        winner_rd, loser_rd = rdHistory_df.loc[timestamp_rd,
                                                [winner_id,loser_id]]
        wp = winProbG(winner_rating, winner_rd, loser_rating, loser_rd)
        return winner_rating, winner_rd, loser_rating, loser_rd, wp
    except KeyError:
        return np.nan, np.nan, np.nan, np.nan,np.nan

In [15]:
apply_glicko2 = matches[in_g2][0:100].apply(lambda x: 
                     get_recent_rating_rd_wp_lambda(x['tourney_date'],
                                                   x['winner_id'],
                                                   x['loser_id'],
                                                   g2_rh, g2_rdh),axis=1)

In [22]:
%%time
apply_glicko2 = matches[in_g2].apply(lambda x: 
                     get_recent_rating_rd_wp_lambda(x['tourney_date'],
                                                   x['winner_id'],
                                                   x['loser_id'],
                                                   g2_rh, g2_rdh),axis=1)

CPU times: user 7min 19s, sys: 7.56 s, total: 7min 26s
Wall time: 7min 21s


The above requires 7 minutes to resolve.  I will save into a dataframe for quick access to the results

,tourney_date,winner_id,loser_id,tourney_id
0,1877-07-09,113987,114149,1877-540
1,1877-07-09,113987,113999,1877-540
2,1877-07-09,113987,113963,1877-540
3,1877-07-09,113963,114158,1877-540
4,1877-07-09,113999,113953,1877-540
...,...,...,...,...
876962,2022-12-26,208168,104581,2022-M-ITF-TUN-63A-2022
876964,2022-12-26,207796,209911,2022-M-ITF-TUN-63A-2022
876965,2022-12-26,207834,208866,2022-M-ITF-TUN-63A-2022
876973,2022-12-26,207987,208519,2022-M-ITF-TUN-63A-2022


In [13]:
# def get_recent_rating_rd_wp(x, ratingsHistory_df, rdHistory_df):
#     """Return the most recent ratings and rating deviations for the competitors 
#     as well as the a priori win probability for the eventual winner according to 
#         the Glicko-2 rating system. **kwargs : ratingsHistory_df,rdHistory_df"""
#     tourney_date = x['tourney_date']
#     winner_id = x['winner_id']
#     loser_id = x['loser_id']
#     try:
#         if not type(tourney_date) == pd._libs.tslibs.timestamps.Timestamp:
#             tourney_date = pd.to_datetime(tourney_date)
#         # for robustness, assuming that rd and ratings may have different timestamps
#         timestamp_rh = max(ratingsHistory_df.index[ratingsHistory_df.index<=tourney_date])
#         timestamp_rd = max(rdHistory_df.index[rdHistory_df.index<=tourney_date])
#         winner_rating, loser_rating = ratingsHistory_df.loc[timestamp_rh,[winner_id,loser_id]]
#         winner_rd, loser_rd = rdHistory_df.loc[timestamp_rd,[winner_id,loser_id]]
#         wp = winProbG(winner_rating, winner_rd, loser_rating, loser_rd)
#         return winner_rating, winner_rd, loser_rating, loser_rd, wp
#     except KeyError:
#         return np.nan, np.nan, np.nan, np.nan,np.nan

###### Elo application

ValueError: max() arg is an empty sequence

In [29]:
padRow = pd.DataFrame({col: 1500.0 for col in ratingsHistory_df.columns}, index = [pd.Timestamp('1877-07-09T00')])
padRow

,131500,131584,131866,131867,131869,131873,131876,131879,131881,131884,...,130340,130355,130457,130493,130542,130550,130552,130553,130655,130767
1877-07-09,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,...,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0


In [30]:
ratingsHistory_df = pd.concat([padRow,ratingsHistory_df],axis=0)

In [ ]:
we_le_pw = matches.apply(lambda x:
                           get_recent_rating_wp(
                               ratingsHistory_df, 
                               x['tourney_date'], 
                               x['winner_id'], 
                               x['loser_id']),
                            axis=1)

In [ ]:
we_le_pw_df = pd.DataFrame.from_records(we_le_pw, columns = ['winner_elo','loser_elo','win_prob'])

In [ ]:
we_le_pw_df

In [ ]:
singles_r_wp = pd.concat([singles_matches,we_le_pw_df],axis = 1)

In [ ]:
singles_r_wp

How well does the elo prediction match with reality?

In [ ]:
singles_r_wp['win_prob'].mean() # the higher elo-rated player wins %56.4 of the time


In [ ]:
singles_r_wp[singles_r_wp['tourney_level']== 'G']['win_prob'].mean() 
# In grand-slam events, the higher rated player wins 58.1% of the time

In [ ]:
singles_r_wp[singles_r_wp['tourney_level']== 'G']

In [ ]:
singles_r_wp['tourney_level'].unique()

In [ ]:
np.mean(singles_r_wp[(singles_r_wp['winner_rank'].isna() == False) & (singles_r_wp['loser_rank'].isna() == False)]['winner_rank'] <singles_r_wp[(singles_r_wp['winner_rank'].isna() == False) & (singles_r_wp['loser_rank'].isna() == False)]['loser_rank'])

Of the ranked matches (both players have a ranking), 64.8% of the winners have a higher ranking.

In [ ]:
np.mean(singles_r_wp[(singles_r_wp['winner_rank'].isna() == False) & (singles_r_wp['loser_rank'].isna() == False)]['loser_elo'] <singles_r_wp[(singles_r_wp['winner_rank'].isna() == False) & (singles_r_wp['loser_rank'].isna() == False)]['winner_elo'])

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
sns.displot(data = singles_r_wp, x = 'win_prob').set(title = 'Histogram of win_prob')

In [ ]:
sns.displot(data = singles_r_wp, x = 'win_prob').set(
    title = 'Histogram of win_prob: elo with 1yr rating periods')
plt.axvline(.5,0,17500, c= 'grey', alpha = .5)
plt.grid()
plt.show()
plt.savefig('../Assets/Elo_1yr_hist',bbox_inches = 'tight')

Above we can see the distribution of win_prob for the winners.  Note the spike at win_prob = .5, which indicates the number of matches between players that had not been rated in the prior epoch and thus have the default rating of 1500.  It is promising that upon visual inspection, the number of wins with win_prob .4 is only slightly above the number of wins with win_prob of .6.  Likewise there are in the ballpark of 25% of the number of wins with a prob of .2 compared to .8.  However, there does appear to be a gradual overestimation of the win probabilities at the extreme ends of the scale.  For instance, at .9 win prob there appears to be only around 3x the number of wins as at .1 win probability.  The model is perhaps not fully picking up the true volatilities in match performance or accounting for the uncertainty of the players' mean ability.

In [ ]:
singles_matches

In [ ]:
# check that elos are not 1500...

In [ ]:
singles_matches[1:10].shape,matches[1:10].shape

In [ ]:
matches.dtypes

In [ ]:
testDf

In [26]:
matches[1:10].apply(
    lambda x: get_recent_rating_wp(ratingsHistory_df, x['tourney_date'], 
                                   x['winner_id'], x['loser_id']),axis=1)

ValueError: max() arg is an empty sequence

In [ ]:
matches[1:10]

In [28]:
ratingsHistory_df

,131500,131584,131866,131867,131869,131873,131876,131879,131881,131884,...,130340,130355,130457,130493,130542,130550,130552,130553,130655,130767
1878-07-09,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,...,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
1879-07-09,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,...,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
1880-07-08,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,...,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
1881-07-08,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,...,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
1882-07-08,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,...,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-06-05,1500.000000,1502.609587,1473.301284,1580.784232,1503.281009,1467.575422,1491.136469,1442.915338,1481.686142,1485.277608,...,1490.231718,1446.916587,1491.681934,1500.000000,1491.945372,1443.751284,1475.091242,1497.078707,1490.832406,1450.854843
2020-06-04,1500.000000,1502.609587,1489.892882,1574.674949,1503.281009,1467.575422,1491.136469,1442.915338,1481.686142,1485.277608,...,1490.231718,1446.916587,1491.681934,1500.000000,1491.945372,1443.751284,1466.854386,1497.078707,1490.832406,1430.651881
2021-06-04,1500.000000,1502.609587,1482.284277,1564.924107,1503.281009,1467.575422,1491.136469,1442.915338,1481.686142,1485.277608,...,1490.231718,1446.916587,1491.681934,1500.000000,1491.945372,1443.751284,1466.854386,1497.078707,1490.832406,1430.651881
2022-06-04,1486.770029,1502.609587,1457.902794,1578.157808,1503.281009,1467.575422,1491.136469,1442.915338,1481.686142,1485.277608,...,1490.231718,1446.916587,1491.681934,1493.325776,1491.945372,1443.751284,1466.854386,1497.078707,1490.832406,1430.651881


In [27]:
testDf['winner_elo','loser_elo','predicted_wins'] = matches[1:10].apply(
    lambda x: get_recent_rating_wp(ratingsHistory_df, x['tourney_date'], 
                                   x['winner_id'], x['loser_id']),axis=1)

ValueError: max() arg is an empty sequence

In [ ]:
testDf = singles_matches[1:10]
testDf['winner_elo','loser_elo','predicted_wins'] = matches[1:10].apply(
    lambda x: get_recent_rating_wp(ratingsHistory_df, x['tourney_date'], 
                                   x['winner_id'], x['loser_id']),axis=1)


In [ ]:
[1,2,3][0:-1]

In [ ]:
ratingsHistory_df

In [ ]:
singles_matches

In [ ]:
ratingsHistory_df.columns

In [ ]:
set(list(matches['winner_id'].unique()) + list(matches['loser_id'].unique()))

In [ ]:
set(ratingsHistory_df) - 

In [ ]:
singles_matches[['winner_elo','loser_elo','predicted_wins']] = matches[['tourney_date','winner_id','loser_id']].apply(lambda x: 
get_recent_rating_wp(ratingsHistory_df, x['tourney_date'], x['winner_id'], x['loser_id']),axis=1)

In [ ]:
ratingsHistory_df